# Scraping CoinAfrique — Utilisant selenium
Scraping des 4 catégories avec une fonction générique et une base de données unifiée.

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialiser Chrome en mode headless
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

## Fonction de scraping générique

In [3]:
def scrape_categorie(driver, slug, nb_pages=9):
    """
    Scrape les annonces d'une catégorie CoinAfrique sur plusieurs pages.

    Args:
        driver   : instance Selenium WebDriver
        slug     : identifiant de la catégorie (ex: 'vetements-homme')
        nb_pages : nombre de pages à scraper

    Returns:
        pd.DataFrame avec les colonnes : categorie, nom, prix, adresse, image_lien
    """
    base_url = f'https://sn.coinafrique.com/categorie/{slug}'
    data = []

    for page in range(1, nb_pages + 1):
        driver.get(f'{base_url}?page={page}')
        time.sleep(2)

        containers = driver.find_elements(By.CSS_SELECTOR, 'div.col.s6.m4.l3')

        for container in containers:
            try:
                nom        = container.find_element(By.CSS_SELECTOR, 'a[title]').get_attribute('title')
                prix       = container.find_element(By.CSS_SELECTOR, 'p.ad__card-price').text
                adresse    = container.find_element(By.CSS_SELECTOR, 'p.ad__card-location').text
                image_lien = container.find_element(By.CSS_SELECTOR, 'img.ad__card-img').get_attribute('src')

                data.append({
                    'categorie'  : slug,
                    'nom'        : nom,
                    'prix'       : prix,
                    'adresse'    : adresse,
                    'image_lien' : image_lien
                })
            except Exception:
                pass

        print(f'[{slug}] Page {page}/{nb_pages} — {len(data)} annonces collectées')

    return pd.DataFrame(data)

## Scraping des 4 catégories + stockage dans une base SQLite unifiée

In [4]:
CATEGORIES = [
    'vetements-homme',
    'chaussures-homme',
    'vetements-enfants',
    'chaussures-enfants',
]

# Connexion à la base unifiée
conn = sqlite3.connect('coinafrique.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS annonces (
        categorie  TEXT,
        nom        TEXT,
        prix       TEXT,
        adresse    TEXT,
        image_lien TEXT
    )
''')
conn.commit()

# Scraper et insérer chaque catégorie
for slug in CATEGORIES:
    print(f'\n--- Catégorie : {slug} ---')
    df = scrape_categorie(driver, slug, nb_pages=9)
    df.to_sql('annonces', conn, if_exists='append', index=False)
    print(f'=> {len(df)} annonces insérées pour "{slug}"')

driver.quit()
print('\nScraping terminé.')


--- Catégorie : vetements-homme ---
[vetements-homme] Page 1/9 — 84 annonces collectées
[vetements-homme] Page 2/9 — 168 annonces collectées
[vetements-homme] Page 3/9 — 252 annonces collectées
[vetements-homme] Page 4/9 — 336 annonces collectées
[vetements-homme] Page 5/9 — 420 annonces collectées
[vetements-homme] Page 6/9 — 504 annonces collectées
[vetements-homme] Page 7/9 — 504 annonces collectées
[vetements-homme] Page 8/9 — 588 annonces collectées
[vetements-homme] Page 9/9 — 672 annonces collectées
=> 672 annonces insérées pour "vetements-homme"

--- Catégorie : chaussures-homme ---
[chaussures-homme] Page 1/9 — 84 annonces collectées
[chaussures-homme] Page 2/9 — 168 annonces collectées
[chaussures-homme] Page 3/9 — 252 annonces collectées
[chaussures-homme] Page 4/9 — 336 annonces collectées
[chaussures-homme] Page 5/9 — 420 annonces collectées
[chaussures-homme] Page 6/9 — 504 annonces collectées
[chaussures-homme] Page 7/9 — 588 annonces collectées
[chaussures-homme] Page 

## Vérification des données

In [5]:
df_all = pd.read_sql_query('SELECT * FROM annonces', conn)

print('Shape :', df_all.shape)
print('\nRépartition par catégorie :')
print(df_all['categorie'].value_counts())
print('\nValeurs manquantes :')
print(df_all.isna().sum())
print('\nDoublons :', df_all.duplicated().sum())

df_all.head()

Shape : (2767, 5)

Répartition par catégorie :
categorie
vetements-enfants     756
vetements-homme       672
chaussures-homme      672
chaussures-enfants    667
Name: count, dtype: int64

Valeurs manquantes :
categorie     0
nom           0
prix          0
adresse       0
image_lien    0
dtype: int64

Doublons : 0


,categorie,nom,prix,adresse,image_lien
0,vetements-homme,Nuasame pour homme,20 000CFA,"location_on\nFass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5758109_u...
1,vetements-homme,Djalabe homme,8 500CFA,"location_on\nFass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5737158_u...
2,vetements-homme,Abaya homme,10 000CFA,"location_on\nFass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5779842_u...
3,vetements-homme,Abaya homme,10 000CFA,"location_on\nFass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5779835_u...
4,vetements-homme,Ensembles Jalabas Homme,9 000CFA,"location_on\nDakar, Sénégal",https://images.coinafrique.com/thumb_5776991_u...


In [6]:
df_all.tail()

,categorie,nom,prix,adresse,image_lien
2762,chaussures-enfants,Chaussures enfant,Prix sur demande,"location_on\nPoint E, Dakar, Sénégal",https://images.coinafrique.com/thumb_792041_up...
2763,chaussures-enfants,Chaussures enfant,Prix sur demande,"location_on\nPoint E, Dakar, Sénégal",https://images.coinafrique.com/thumb_792038_up...
2764,chaussures-enfants,Claquettes enfants,Prix sur demande,location_on\nSénégal,https://images.coinafrique.com/thumb_781661_up...
2765,chaussures-enfants,Chaussures enfant,Prix sur demande,"location_on\nGuédiawaye, Dakar, Sénégal",https://images.coinafrique.com/thumb_750022_up...
2766,chaussures-enfants,Chaussures enfant,Prix sur demande,"location_on\nGuédiawaye, Dakar, Sénégal",https://images.coinafrique.com/thumb_750016_up...


## Pas de données manquantes, pas de doublons donc pas de netoyage à faire.
## Toutes les données stockées dans la bd (sqlite) "coinafrique.db" 